### Workflow 💡💡


## CUDA

### Basics

- Threads, Blocks, Grids   ✅
- Memory Hierarchy (global, shared, and local memory)✅

- Implement parallel versions of classic algorithms:  
  - Matrix multiplication   ✅
  - Reduction
  - Sorting  

### Advanced
  - cuBLAS
  - cuDNN
  - Thrust
  -Multi-GPU programming and CUDA
 streams for concurrent execution


### Project Ideas
- Optimize an existing ML model’s inference or training pipeline using CUDA. ⚙️  
Custom Layers  
- Implement custom CUDA kernels for specific operations in an ML pipeline:  
  - Novel activation function  
  - Custom loss function  

  
  - Model parallelism
  - Distributed training
  - Quantization
  - Optimize training/inference using CUDA.

In [ ]:
!python --version
!nvcc --version
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

Python 3.11.11
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpw7rzml6a".


In [ ]:
!apt-get install -y cuda-toolkit-11-2  # Install CUDA toolkit (adjust version if needed)
!nvcc --version  # Verify CUDA compiler installation
!pip install pycuda

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package cuda-toolkit-11-2
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.8/92.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 7.8 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1.2-cp311-cp311-linux_x86_64.whl size=660362 sha256=0654fbc7f8d4469af246bacfee13763eee4f8b3117c6574ce883c11f44cc06a7
  Stored in directory: /root/.cache/pip/wheels/76/66/50/c65e6116d7e0e16abe0f7c19b50327f76724ccfefbdc61a1b9
Succes

#CUDA program surface level runtime:

### copy input from host to device
### load GPU program and execute using the transferred on-device data
### copy results from device back to host so you can display/use it some

### Cuda Naming

#### `h_A` - refers to host (CPU) for variable name “A”
#### `d_A` - refers to device (GPU) for variable name “A”
#### `__global__`- visible globally, meaning the CPU or host can call these global functions.
#### `__device__`
#### `__host__`


### Memory Allocation in GPU VRAM

### `cudaMalloc` - Global memory on GPU itself




`    float *d_a, *d_b, *d_c; // float array which is a pointer on the device for a , b and c`

    cudaMalloc(&d_a, N*N*sizeof(float));
    cudaMalloc(&d_b, N*N*sizeof(float));
    cudaMalloc(&d_c, N*N*sizeof(float));`

In [ ]:
%%writefile vector_add.cu
// vector_add.cu
__global__ void vector_add(float *a, float *b, float *c, int n) {
  int idx = threadIdx.x + blockIdx.x * blockDim.x;
  if (idx < n) {
    c[idx] = a[idx] + b[idx];
  }
}

Overwriting vector_add.cu


Using `nvcc` to compile the `.cu` file into a shared object `(.so)` file:

In [ ]:
!nvcc -o vector_add.cu --shared -Xcompiler -fPIC vector_add.cu

In [ ]:
!nvcc -ptx vector_add.cu -o vector_add.ptx

#### Loading compiled kernel using PyCUDA in python

In [ ]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.8/92.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 7.5 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1.2-cp311-cp311-linux_x86_64.whl size=660362 sha256=6b9d10fa015d96885d9c5174298cd3c664ad41e239cb6dacd76e56e2aceb6aad
  Stored in directory: /root/.cache/pip/wheels/76/66/50/c65e6116d7e0e16abe0f7c19b50327f76724ccfefbdc61a1b9
Successfully built pycuda


In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np

# Load the precompiled PTX file
with open("vector_add.ptx", "r") as f:
    ptx_code = f.read()

# Create a SourceModule from the PTX code
mod = SourceModule(ptx_code)

# Get the kernel function
vector_add = mod.get_function("vector_add")

# Define input vectors
n = 10**6  # Increase vector size for better GPU utilization
a = np.random.randn(n).astype(np.float32)
b = np.random.randn(n).astype(np.float32)
c = np.zeros_like(a)

# Allocate memory on the GPU
a_gpu = cuda.mem_alloc(a.nbytes)
b_gpu = cuda.mem_alloc(b.nbytes)
c_gpu = cuda.mem_alloc(c.nbytes)

# Copy data to the GPU
cuda.memcpy_htod(a_gpu, a)
cuda.memcpy_htod(b_gpu, b)

# Adjust block and grid sizes
block_size = min(256, n)  # Use the smaller of 256 or n
grid_size = (n + block_size - 1) // block_size

# Launch the kernel
vector_add(a_gpu, b_gpu, c_gpu, np.int32(n), block=(block_size, 1, 1), grid=(grid_size, 1))

# Copy the result back to the host
cuda.memcpy_dtoh(c, c_gpu)

# Verify the result
print("A:", a[:10])  # Print only the first 10 elements for brevity
print("B:", b[:10])
print("C (A + B):", c[:10])

CompileError: nvcc compilation of /tmp/tmpnh198ou8/kernel.cu failed
[command: nvcc --cubin -arch sm_80 -I/usr/local/lib/python3.11/dist-packages/pycuda/cuda kernel.cu]
[stderr:
kernel.cu(10): error: expected a declaration
  .version 8.5
  ^

kernel.cu(38): error: unrecognized token
   @%p1 bra $L__BB0_2;
   ^

kernel.cu(43): warning #12-D: parsing restarts here after previous syntax error
   cvta.to.global.u64 %rd7, %rd2;
                                ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

kernel.cu(44): error: this declaration has no storage class or type specifier
   add.s64 %rd8, %rd7, %rd5;
   ^

kernel.cu(44): error: expected a ";"
   add.s64 %rd8, %rd7, %rd5;
      ^

kernel.cu(45): error: this declaration has no storage class or type specifier
   ld.global.f32 %f1, [%rd8];
   ^

kernel.cu(45): error: expected a ";"
   ld.global.f32 %f1, [%rd8];
     ^

kernel.cu(46): error: this declaration has no storage class or type specifier
   ld

In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np
import time

# Load the CUDA kernel
with open("vector_add.cu", "rb") as f:
    mod = SourceModule(f.read())

# Get the kernel function
vector_add = mod.get_function("vector_add")

# Define input vectors
n = 10**6  # Increase vector size for better GPU utilization
a = np.random.randn(n).astype(np.float32)
b = np.random.randn(n).astype(np.float32)
c = np.zeros_like(a)

# Allocate memory on the GPU
a_gpu = cuda.mem_alloc(a.nbytes)
b_gpu = cuda.mem_alloc(b.nbytes)
c_gpu = cuda.mem_alloc(c.nbytes)

# Copy data to the GPU
cuda.memcpy_htod(a_gpu, a)
cuda.memcpy_htod(b_gpu, b)

# Adjust block and grid sizes
block_size = min(256, n)  # Use the smaller of 256 or n
grid_size = (n + block_size - 1) // block_size

# Launch the kernel
start = time.time()
vector_add(a_gpu, b_gpu, c_gpu, np.int32(n), block=(block_size, 1, 1), grid=(grid_size, 1))
end = time.time()
print(f"Kernel execution time: {end - start:.4f} seconds")

# Copy the result back to the host
cuda.memcpy_dtoh(c, c_gpu)

# Verify the result
print("A:", a[:10])  # Print only the first 10 elements for brevity
print("B:", b[:10])
print("C (A + B):", c[:10])

CompileError: nvcc compilation of /tmp/tmpnoqlc3xf/kernel.cu failed
[command: nvcc --cubin -arch sm_80 -I/usr/local/lib/python3.11/dist-packages/pycuda/cuda kernel.cu]
[stderr:
kernel.cu(2): warning #1654-D: too many characters in character literal -- extra leading characters ignored
  b'// vector_add.cu\n__global__ void vector_add(float *a, float *b, float *c, int n) {\n  int idx = threadIdx.x + blockIdx.x * blockDim.x;\n  if (idx < n) {\n    c[idx] = a[idx] + b[idx]\n  }\n}\n'
   ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

kernel.cu(2): error: this declaration has no storage class or type specifier
  b'// vector_add.cu\n__global__ void vector_add(float *a, float *b, float *c, int n) {\n  int idx = threadIdx.x + blockIdx.x * blockDim.x;\n  if (idx < n) {\n    c[idx] = a[idx] + b[idx]\n  }\n}\n'
  ^

kernel.cu(2): error: expected a ";"
  b'// vector_add.cu\n__global__ void vector_add(float *a, float *b, float *c, int n) {\n  int idx = threadIdx.x +

In [ ]:
!apt-get install -y cuda-toolkit-11-2  # Install CUDA toolkit
!nvcc --version  # Verify installation

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package cuda-toolkit-11-2
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
%%writefile idxing.cu

#include <stdio.h>

// vector_add.cu

__global__ void whoami(void) {
    int block_id =
        blockIdx.x +    // apartment number on this floor (points across)
        blockIdx.y * gridDim.x +    // floor number in this building (rows high)
        blockIdx.z * gridDim.x * gridDim.y;   //dims of x and y on the grid

    int block_offset =
        block_id * // times our apartment number
        blockDim.x * blockDim.y * blockDim.z; // total threads per block (people per apartment)
                                              // how big that block is
    int thread_offset =
        threadIdx.x +                         // which thread is it within that block
        threadIdx.y * blockDim.x +
        threadIdx.z * blockDim.x * blockDim.y;

    int id = block_offset + thread_offset; // global person id in the entire apartment complex

    printf("%04d | Block(%d %d %d) = %3d | Thread(%d %d %d) = %3d\n",
        id,
        blockIdx.x, blockIdx.y, blockIdx.z, block_id,
        threadIdx.x, threadIdx.y, threadIdx.z, thread_offset);
    // printf("blockIdx.x: %d, blockIdx.y: %d, blockIdx.z: %d, threadIdx.x: %d, threadIdx.y: %d, threadIdx.z: %d\n", blockIdx.x, blockIdx.y, blockIdx.z, threadIdx.x, threadIdx.y, threadIdx.z);
}

int main(int argc, char **argv) {
    const int b_x = 2, b_y = 3, b_z = 4; // shape of the block, think shape of matrix in for eg np
    const int t_x = 4, t_y = 4, t_z = 4; // the max warp size is 32, so (shape of grid)
    // we will get 2 warp of 32 threads per block

    int blocks_per_grid = b_x * b_y * b_z;
    int threads_per_block = t_x * t_y * t_z;

    printf("%d blocks/grid\n", blocks_per_grid);
    printf("%d threads/block\n", threads_per_block);
    printf("%d total threads\n", blocks_per_grid * threads_per_block);

    dim3 blocksPerGrid(b_x, b_y, b_z); // 3d cube of shape 2*3*4 = 24
    dim3 threadsPerBlock(t_x, t_y, t_z); // 3d cube of shape 4*4*4 = 64

    whoami<<<blocksPerGrid, threadsPerBlock>>>(); // passing our parameters , calling our function basically
    cudaDeviceSynchronize();
}

Overwriting idxing.cu


In [ ]:
!nvcc idxing.cu -o idxing

In [ ]:
!./idxing

24 blocks/grid
64 threads/block
1536 total threads


In [ ]:
!nvcc idxing.cu -cubin -o idxing.cubin
!nvdisasm idxing.cubin


	.headerflags	@"EF_CUDA_TEXMODE_UNIFIED EF_CUDA_64BIT_ADDRESS EF_CUDA_SM52 EF_CUDA_VIRTUAL_SM(EF_CUDA_SM52)"
	.elftype	@"ET_EXEC"


//--------------------- .nv.info                  --------------------------
	.section	.nv.info,"",@"SHT_CUDA_INFO"
	.align	4


	//----- nvinfo : EIATTR_REGCOUNT
	.align		4
        /*0000*/ 	.byte	0x04, 0x2f
        /*0002*/ 	.short	(.L_2 - .L_1)
	.align		4
.L_1:
        /*0004*/ 	.word	index@(_Z6whoamiv)
        /*0008*/ 	.word	0x00000014


	//----- nvinfo : EIATTR_MIN_STACK_SIZE
	.align		4
.L_2:
        /*000c*/ 	.byte	0x04, 0x12
        /*000e*/ 	.short	(.L_4 - .L_3)
	.align		4
.L_3:
        /*0010*/ 	.word	index@(_Z6whoamiv)
        /*0014*/ 	.word	0x00000028


	//----- nvinfo : EIATTR_FRAME_SIZE
	.align		4
.L_4:
        /*0018*/ 	.byte	0x04, 0x11
        /*001a*/ 	.short	(.L_6 - .L_5)
	.align		4
.L_5:
        /*001c*/ 	.word	index@(_Z6whoamiv)
        /*0020*/ 	.word	0x00000028


	//----- nvinfo : EIATTR_MIN_STACK_SIZE
	.align		4
.L_6:
        /*0024*

In [ ]:
!nvcc -G idxing.cu -o idxing


In [ ]:
%%writefile thread_idx.cu
include <stdio.h>

__global__ void whoami(void) {
    int id = threadIdx.x + threadIdx.y * blockDim.x + threadIdx.z * blockDim.x * blockDim.y;
    printf("Thread in block (%d,%d,%d): id = %d\n", blockIdx.x, blockIdx.y, blockIdx.z, id);
}

int main() {
    dim3 blocks(2, 2, 1);
    dim3 threads(4, 4, 1);
    whoami<<<blocks, threads>>>();
    cudaDeviceSynchronize();
    return 0;
}


Writing thread_idx.cu


In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda_runtime.h>

#define N 10000000  // Vector size = 10 million
#define BLOCK_SIZE 256

// Example:
// A = [1, 2, 3, 4, 5]
// B = [6, 7, 8, 9, 10]
// C = A + B = [7, 9, 11, 13, 15]

// CPU vector addition
void vector_add_cpu(float *a, float *b, float *c, int n) {
    for (int i = 0; i < n; i++) {
        c[i] = a[i] + b[i];
    }
}

// CUDA kernel for vector addition
__global__ void vector_add_gpu(float *a, float *b, float *c, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n) {
        c[i] = a[i] + b[i];
    }
}

// Initialize vector with random values
void init_vector(float *vec, int n) {
    for (int i = 0; i < n; i++) {
        vec[i] = (float)rand() / RAND_MAX;
    }
}

// Function to measure execution time
double get_time() {
    struct timespec ts;
    clock_gettime(CLOCK_MONOTONIC, &ts);
    return ts.tv_sec + ts.tv_nsec * 1e-9;
}

int main() {
    float *h_a, *h_b, *h_c_cpu, *h_c_gpu;
    float *d_a, *d_b, *d_c;
    size_t size = N * sizeof(float);

    // Allocate host memory
    h_a = (float*)malloc(size);
    h_b = (float*)malloc(size);
    h_c_cpu = (float*)malloc(size);
    h_c_gpu = (float*)malloc(size);

    // Initialize vectors
    srand(time(NULL));
    init_vector(h_a, N);
    init_vector(h_b, N);

    // Allocate device memory
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);

    // Copy data to device
    cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice);

    // Define grid and block dimensions
    int num_blocks = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;
    // N = 1024, BLOCK_SIZE = 256, num_blocks = 4
    // (N + BLOCK_SIZE - 1) / BLOCK_SIZE = ( (1025 + 256 - 1) / 256 ) = 1280 / 256 = 4 rounded

    // Warm-up runs
    printf("Performing warm-up runs...\n");
    for (int i = 0; i < 3; i++) {
        vector_add_cpu(h_a, h_b, h_c_cpu, N);
        vector_add_gpu<<<num_blocks, BLOCK_SIZE>>>(d_a, d_b, d_c, N);
        cudaDeviceSynchronize();
    }

    // Benchmark CPU implementation
    printf("Benchmarking CPU implementation...\n");
    double cpu_total_time = 0.0;
    for (int i = 0; i < 20; i++) {
        double start_time = get_time();
        vector_add_cpu(h_a, h_b, h_c_cpu, N);
        double end_time = get_time();
        cpu_total_time += end_time - start_time;
    }
    double cpu_avg_time = cpu_total_time / 20.0;

    // Benchmark GPU implementation
    printf("Benchmarking GPU implementation...\n");
    double gpu_total_time = 0.0;
    for (int i = 0; i < 20; i++) {
        double start_time = get_time();
        vector_add_gpu<<<num_blocks, BLOCK_SIZE>>>(d_a, d_b, d_c, N);
        cudaDeviceSynchronize();
        double end_time = get_time();
        gpu_total_time += end_time - start_time;
    }
    double gpu_avg_time = gpu_total_time / 20.0;

    // Print results
    printf("CPU average time: %f milliseconds\n", cpu_avg_time*1000);
    printf("GPU average time: %f milliseconds\n", gpu_avg_time*1000);
    printf("Speedup: %fx\n", cpu_avg_time / gpu_avg_time);

    // Verify results (optional)
    cudaMemcpy(h_c_gpu, d_c, size, cudaMemcpyDeviceToHost);
    bool correct = true;
    for (int i = 0; i < N; i++) {
        if (fabs(h_c_cpu[i] - h_c_gpu[i]) > 1e-5) {
            correct = false;
            break;
        }
    }
    printf("Results are %s\n", correct ? "correct" : "incorrect");

    // Free memory
    free(h_a);
    free(h_b);
    free(h_c_cpu);
    free(h_c_gpu);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}

In [ ]:
!pip install jupyterlab-lsp
!pip install jupyter-lsp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 120.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.8/385.8 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 6.1.12
    Uninstalling jupyter-client-6.1.12:
      Successfully uninstalled jupyter-client-6.1.12
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 5.5.6
    Uninstalling ip

In [ ]:
!pip install jupyterlab
!jupyter serverextension enable --py jupyterlab

Enabling: jupyterlab
- Writing config: /root/.jupyter
    - Validating...
      jupyterlab 4.3.5 OK


In [ ]:
from google.colab import output
output.serve_kernel_port_as_window(8888)
!jupyter lab --port=8888 --no-browser --ip=0.0.0.0

Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

[I 2025-02-09 00:06:30.845 ServerApp] ipyparallel | extension was successfully linked.
[I 2025-02-09 00:06:30.845 ServerApp] jupyter_lsp | extension was successfully linked.
[I 2025-02-09 00:06:30.849 ServerApp] jupyter_server_terminals | extension was successfully linked.
[I 2025-02-09 00:06:30.853 ServerApp] jupyterlab | extension was successfully linked.
[W 2025-02-09 00:06:30.855 NotebookApp] 'allow_root' has moved from NotebookApp to ServerApp. This config will be passed to ServerApp. Be sure to update your config before our next release.
[W 2025-02-09 00:06:30.855 NotebookApp] 'allow_root' has moved from NotebookApp to ServerApp. This config will be passed to ServerApp. Be sure to update your config before our next release.
[W 2025-02-09 00:06:30.855 NotebookApp] 'port_retries' has moved from NotebookApp to ServerApp. This config will be passed to ServerApp. Be sure to update your config before our next release.
[W 2025-02-09 00:06:30.855 NotebookApp] 'iopub_data_rate_limit' has 

In [ ]:
# @title Texto de título predeterminado

%%writefile vector_add.cu


#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda_runtime.h>

#define N 10000000  // Vector size = 10 million
#define BLOCK_SIZE 256

// Example:
// A = [1, 2, 3, 4, 5]
// B = [6, 7, 8, 9, 10]
// C = A + B = [7, 9, 11, 13, 15]

// CPU vector addition

void vector_add_cpu(float *a, float *b, float *c, int n) {
  for (int i = 0; i < n; i++) {
    c[i] = a[i] + b[i]; //what we looked at b4
  }
}


// cuda kernel for vector addition

__global__ void vector_add_gpu(float *a, float *b, float *c, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n) {            // instead of having a forloop, what were doing here is unrolling this loop
        c[i] = a[i] + b[i]; // and distribuiting them across different blocks, parallelizing the operation and distribuiting them separately ]

    }
}


//init our vector

void init_vector(float *vec, int n) {
  for (int i = 0; i < n; i++) {
    vec[i] = (float)rand() / RAND_MAX;
  }
}

// Function to measure execution time - to benchmark perf


double get_time() {
    struct timespec ts;
    clock_gettime(CLOCK_MONOTONIC, &ts);
    return ts.tv_sec + ts.tv_nsec * 1e-9;
}

int main() {
    float *h_a, *h_b, *h_c_cpu, *h_c_gpu;
    float *d_a, *d_b, *d_c;
    size_t size = N * sizeof(float);

    // Allocate host memory
    h_a = (float*)malloc(size);
    h_b = (float*)malloc(size);
    h_c_cpu = (float*)malloc(size);
    h_c_gpu = (float*)malloc(size);

    // Initialize vectors
    srand(time(NULL));
    init_vector(h_a, N);
    init_vector(h_b, N);

    // Allocate device memory
    cudaMalloc(&d_a, size); // device pointer, memry address
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);

    // Copy data to device
    cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice); // CPU moving to GPU

    cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice);

    // Define grid and block dimensions
    int num_blocks = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;
    // N = 1024, BLOCK_SIZE = 256, num_blocks = 4
    // (N + BLOCK_SIZE - 1) / BLOCK_SIZE = ( (1025 + 256 - 1) / 256 ) = 1280 / 256 = 4 rounded

    // Warm-up runs
    printf("Performing warm-up runs...\n");
    for (int i = 0; i < 3; i++) {
        vector_add_cpu(h_a, h_b, h_c_cpu, N);
        vector_add_gpu<<<num_blocks, BLOCK_SIZE>>>(d_a, d_b, d_c, N);
        cudaDeviceSynchronize();
    }

    // Benchmark CPU implementation
    printf("Benchmarking CPU implementation...\n");
    double cpu_total_time = 0.0;
    for (int i = 0; i < 20; i++) {
        double start_time = get_time();
        vector_add_cpu(h_a, h_b, h_c_cpu, N);
        double end_time = get_time();
        cpu_total_time += end_time - start_time;
    }
    double cpu_avg_time = cpu_total_time / 20.0;

    // Benchmark GPU implementation
    printf("Benchmarking GPU implementation...\n");
    double gpu_total_time = 0.0;
    for (int i = 0; i < 20; i++) {
        double start_time = get_time();
        vector_add_gpu<<<num_blocks, BLOCK_SIZE>>>(d_a, d_b, d_c, N);
        cudaDeviceSynchronize();
        double end_time = get_time();
        gpu_total_time += end_time - start_time;
    }
    double gpu_avg_time = gpu_total_time / 20.0;

    // Print results
    printf("CPU average time: %f milliseconds\n", cpu_avg_time*1000);
    printf("GPU average time: %f milliseconds\n", gpu_avg_time*1000);
    printf("Speedup: %fx\n", cpu_avg_time / gpu_avg_time);

    // Verify results (optional)
    cudaMemcpy(h_c_gpu, d_c, size, cudaMemcpyDeviceToHost);
    bool correct = true;
    for (int i = 0; i < N; i++) {
        if (fabs(h_c_cpu[i] - h_c_gpu[i]) > 1e-5) {
            correct = false;
            break;
        }
    }
    printf("Results are %s\n", correct ? "correct" : "incorrect");

    // Free memory
    free(h_a);
    free(h_b);
    free(h_c_cpu);
    free(h_c_gpu);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);


}




UsageError: unrecognized arguments: --language vector_add.cu


In [ ]:
!nvcc vector_add.cu -o vector_add


In [ ]:
!./vector_add


Performing warm-up runs...
Benchmarking CPU implementation...
Benchmarking GPU implementation...
CPU average time: 33.201688 milliseconds
GPU average time: 0.002372 milliseconds
Speedup: 13995.273745x
Results are incorrect


In [ ]:
%%writefile vector_add.cpp
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda_runtime.h>
#include <math.h>

#define N 10000000  // Vector size = 10 million
#define BLOCK_SIZE 256

// Macro for error checking
#define cudaCheckError(call) {                                     \
    cudaError_t err = call;                                        \
    if (err != cudaSuccess) {                                      \
        fprintf(stderr, "CUDA error in %s at %s:%d: %s\n",         \
                #call, __FILE__, __LINE__, cudaGetErrorString(err)); \
        exit(err);                                                 \
    }                                                              \
}

// CPU vector addition
void vector_add_cpu(float *a, float *b, float *c, int n) {
  for (int i = 0; i < n; i++) {
    c[i] = a[i] + b[i];
  }
}

// CUDA kernel for vector addition
__global__ void vector_add_gpu(float *a, float *b, float *c, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;  // how many threads are there / block * no. of blocks
    if (i < n) {
        c[i] = a[i] + b[i];
    }
}

// Initialize vector with random float values between 0 and 1
void init_vector(float *vec, int n) {
  for (int i = 0; i < n; i++) {
    vec[i] = (float)rand() / RAND_MAX;
  }
}

// Function to get the current time in seconds (for benchmarking)
double get_time() {
    struct timespec ts;
    clock_gettime(CLOCK_MONOTONIC, &ts);
    return ts.tv_sec + ts.tv_nsec * 1e-9;
}

int main() {
    float *h_a, *h_b, *h_c_cpu, *h_c_gpu;
    float *d_a, *d_b, *d_c;
    size_t size = N * sizeof(float);

    // Allocate host memory
    h_a = (float*)malloc(size);
    h_b = (float*)malloc(size);
    h_c_cpu = (float*)malloc(size);
    h_c_gpu = (float*)malloc(size);

    // Initialize vectors
    srand(time(NULL));
    init_vector(h_a, N);
    init_vector(h_b, N);

    // Allocate device memory
    cudaCheckError(cudaMalloc(&d_a, size));
    cudaCheckError(cudaMalloc(&d_b, size));
    cudaCheckError(cudaMalloc(&d_c, size));

    // Copy data to device
    cudaCheckError(cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice));
    cudaCheckError(cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice));

    // Define grid and block dimensions
    int num_blocks = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;

    // Warm-up runs
    printf("Performing warm-up runs...\n");
    for (int i = 0; i < 3; i++) {
        vector_add_cpu(h_a, h_b, h_c_cpu, N);
        vector_add_gpu<<<num_blocks, BLOCK_SIZE>>>(d_a, d_b, d_c, N);
        cudaCheckError(cudaGetLastError());
        cudaCheckError(cudaDeviceSynchronize());
    }

    // Benchmark CPU implementation
    printf("Benchmarking CPU implementation...\n");
    double cpu_total_time = 0.0;
    for (int i = 0; i < 20; i++) {
        double start_time = get_time();
        vector_add_cpu(h_a, h_b, h_c_cpu, N);
        double end_time = get_time();
        cpu_total_time += end_time - start_time;
    }
    double cpu_avg_time = cpu_total_time / 20.0;

    // Benchmark GPU implementation
    printf("Benchmarking GPU implementation...\n");
    double gpu_total_time = 0.0;
    for (int i = 0; i < 20; i++) {
        // (Optional) re-copy inputs if needed
        // cudaCheckError(cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice));
        // cudaCheckError(cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice));

        double start_time = get_time();
        vector_add_gpu<<<num_blocks, BLOCK_SIZE>>>(d_a, d_b, d_c, N);
        cudaCheckError(cudaGetLastError());
        cudaCheckError(cudaDeviceSynchronize());
        double end_time = get_time();
        gpu_total_time += end_time - start_time;
    }
    double gpu_avg_time = gpu_total_time / 20.0;

    // Copy GPU result back to host
    cudaCheckError(cudaMemcpy(h_c_gpu, d_c, size, cudaMemcpyDeviceToHost));

    // Print a few sample values for comparison
    printf("First 10 elements:\n");
    for (int i = 0; i < 10; i++) {
        printf("Index %d: CPU = %f, GPU = %f\n", i, h_c_cpu[i], h_c_gpu[i]);
    }

    // Verify results
    bool correct = true;
    for (int i = 0; i < N; i++) {
        if (fabs(h_c_cpu[i] - h_c_gpu[i]) > 1e-5) {
            correct = false;
            break;
        }
    }

    printf("CPU average time: %f milliseconds\n", cpu_avg_time * 1000);
    printf("GPU average time: %f milliseconds\n", gpu_avg_time * 1000);
    printf("Speedup: %fx\n", cpu_avg_time / gpu_avg_time);
    printf("Results are %s\n", correct ? "correct" : "incorrect");

    // Free memory
    free(h_a);
    free(h_b);
    free(h_c_cpu);
    free(h_c_gpu);
    cudaCheckError(cudaFree(d_a));
    cudaCheckError(cudaFree(d_b));
    cudaCheckError(cudaFree(d_c));

    return 0;
}


Writing vector_add.cpp


In [ ]:
!mv vector_add.cpp vector_add.cu

In [ ]:
!nvcc -gencode arch=compute_80,code=sm_80 vector_add.cu -o vector_add



!nvcc -gencode arch=compute_80,code=sm_80 vector_add.cu -o vector_add


In [ ]:
!./vector_add


Performing warm-up runs...
Benchmarking CPU implementation...
Benchmarking GPU implementation...
First 10 elements:
Index 0: CPU = 0.873982, GPU = 0.873982
Index 1: CPU = 1.092135, GPU = 1.092135
Index 2: CPU = 0.329347, GPU = 0.329347
Index 3: CPU = 1.244920, GPU = 1.244920
Index 4: CPU = 0.121245, GPU = 0.121245
Index 5: CPU = 0.936237, GPU = 0.936237
Index 6: CPU = 1.370975, GPU = 1.370975
Index 7: CPU = 1.169952, GPU = 1.169952
Index 8: CPU = 1.097276, GPU = 1.097276
Index 9: CPU = 1.400032, GPU = 1.400032
CPU average time: 33.354476 milliseconds
GPU average time: 0.101068 milliseconds
Speedup: 330.019491x
Results are correct


### Naive Matmul

In [ ]:
%%writefile matmul.cpp

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda_runtime.h>

#define M 256  // Number of rows in A and C
#define K 512   // Number of columns in A and rows in B
#define N 256  // Number of columns in B and C
#define BLOCK_SIZE 32

// Example 3x2 @ 2x4 = 3x4 -> (M x K) @ (K x N) = (M x N)
// A = [[1, 2],
//      [3, 4],
//      [5, 6]]

// B = [[7, 8, 9, 10],
//      [11, 12, 13, 14]]

// C = A * B = [[1*7 + 2*11, 1*8 + 2*12, 1*9 + 2*13, 1*10 + 2*14],
//              [3*7 + 4*11, 3*8 + 4*12, 3*9 + 4*13, 3*10 + 4*14],
//              [5*7 + 6*11, 5*8 + 6*12, 5*9 + 6*13, 5*10 + 6*14]]

// C = [[29, 32, 35, 38],
//      [65, 72, 79, 86],
//      [101, 112, 123, 134]]


// CPU matrix multiplication
void matmul_cpu(float *A, float *B, float *C, int m, int k, int n) {
    for (int i = 0; i < m; i++) {
        for (int j = 0; j < n; j++) {
            float sum = 0.0f;
            for (int l = 0; l < k; l++) {
                sum += A[i * k + l] * B[l * n + j];
            }
            C[i * n + j] = sum;
        }
    }
}

// CUDA kernel for matrix multiplication
__global__ void matmul_gpu(float *A, float *B, float *C, int m, int k, int n) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < m && col < n) {
        float sum = 0.0f;
        for (int l = 0; l < k; l++) {
            sum += A[row * k + l] * B[l * n + col];
        }
        C[row * n + col] = sum;
    }
}

// Initialize matrix with random values
void init_matrix(float *mat, int rows, int cols) {
    for (int i = 0; i < rows * cols; i++) {
        mat[i] = (float)rand() / RAND_MAX;
    }
}

// Function to measure execution time
double get_time() {
    struct timespec ts;
    clock_gettime(CLOCK_MONOTONIC, &ts);
    return ts.tv_sec + ts.tv_nsec * 1e-9;
}

int main() {
    float *h_A, *h_B, *h_C_cpu, *h_C_gpu;
    float *d_A, *d_B, *d_C;
    int size_A = M * K * sizeof(float);
    int size_B = K * N * sizeof(float);
    int size_C = M * N * sizeof(float);

    // Allocate host memory
    h_A = (float*)malloc(size_A);
    h_B = (float*)malloc(size_B);
    h_C_cpu = (float*)malloc(size_C);
    h_C_gpu = (float*)malloc(size_C);

    // Initialize matrices
    srand(time(NULL));
    init_matrix(h_A, M, K);
    init_matrix(h_B, K, N);

    // Allocate device memory
    cudaMalloc(&d_A, size_A);
    cudaMalloc(&d_B, size_B);
    cudaMalloc(&d_C, size_C);

    // Copy data to device
    cudaMemcpy(d_A, h_A, size_A, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size_B, cudaMemcpyHostToDevice);

    // Define grid and block dimensions
    dim3 blockDim(BLOCK_SIZE, BLOCK_SIZE);
    dim3 gridDim((N + BLOCK_SIZE - 1) / BLOCK_SIZE, (M + BLOCK_SIZE - 1) / BLOCK_SIZE);

    // Warm-up runs
    printf("Performing warm-up runs...\n");
    for (int i = 0; i < 3; i++) {
        matmul_cpu(h_A, h_B, h_C_cpu, M, K, N);
        matmul_gpu<<<gridDim, blockDim>>>(d_A, d_B, d_C, M, K, N);
        cudaDeviceSynchronize();
    }

    // Benchmark CPU implementation
    printf("Benchmarking CPU implementation...\n");
    double cpu_total_time = 0.0;
    for (int i = 0; i < 20; i++) {
        double start_time = get_time();
        matmul_cpu(h_A, h_B, h_C_cpu, M, K, N);
        double end_time = get_time();
        cpu_total_time += end_time - start_time;
    }
    double cpu_avg_time = cpu_total_time / 20.0;

    // Benchmark GPU implementation
    printf("Benchmarking GPU implementation...\n");
    double gpu_total_time = 0.0;
    for (int i = 0; i < 20; i++) {
        double start_time = get_time();
        matmul_gpu<<<gridDim, blockDim>>>(d_A, d_B, d_C, M, K, N);
        cudaDeviceSynchronize();
        double end_time = get_time();
        gpu_total_time += end_time - start_time;
    }
    double gpu_avg_time = gpu_total_time / 20.0;

    // Print results
    printf("CPU average time: %f microseconds\n", (cpu_avg_time * 1e6f));
    printf("GPU average time: %f microseconds\n", (gpu_avg_time * 1e6f));
    printf("Speedup: %fx\n", cpu_avg_time / gpu_avg_time);

    // Free memory
    free(h_A);
    free(h_B);
    free(h_C_cpu);
    free(h_C_gpu);
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    return 0;
}

Writing matmul.cpp


In [ ]:
!mv matmul.cpp matmul.cu

In [ ]:
!nvcc -gencode arch=compute_80,code=sm_80 matmul.cu -o matmul


In [ ]:
!./matmul


Performing warm-up runs...
Benchmarking CPU implementation...
Benchmarking GPU implementation...
CPU average time: 138633.033650 microseconds
GPU average time: 52.540550 microseconds
Speedup: 2638.591214x


In [ ]:
%%writefile t_matmul.cpp
#include <cuda_runtime.h>
#include <iostream>
#include <vector>

#define TILE_SIZE 16

__global__ void matrixMultiplyOptimized(float* A, float* B, float* C, int M, int N, int K) {
    __shared__ float sharedA[TILE_SIZE][TILE_SIZE];
    __shared__ float sharedB[TILE_SIZE][TILE_SIZE];

    int bx = blockIdx.x, by = blockIdx.y;
    int tx = threadIdx.x, ty = threadIdx.y;

    int row = by * TILE_SIZE + ty;
    int col = bx * TILE_SIZE + tx;

    float sum = 0.0f;

    for (int tile = 0; tile < (K + TILE_SIZE - 1) / TILE_SIZE; ++tile) {
        if (row < M && tile * TILE_SIZE + tx < K)
            sharedA[ty][tx] = A[row * K + tile * TILE_SIZE + tx];
        else
            sharedA[ty][tx] = 0.0f;

        if (col < N && tile * TILE_SIZE + ty < K)
            sharedB[ty][tx] = B[(tile * TILE_SIZE + ty) * N + col];
        else
            sharedB[ty][tx] = 0.0f;

        __syncthreads();

        for (int k = 0; k < TILE_SIZE; ++k)
            sum += sharedA[ty][k] * sharedB[k][tx];

        __syncthreads();
    }

    if (row < M && col < N)
        C[row * N + col] = sum;
}

int main() {

    // Define matrix dimensions
    const int M = 1024; // Number of rows in A and C
    const int N = 1024; // Number of columns in B and C
    const int K = 1024; // Number of columns in A and rows in B

    // Calculate matrix sizes in bytes
    size_t size_A = M * K * sizeof(float);
    size_t size_B = K * N * sizeof(float);
    size_t size_C = M * N * sizeof(float);

    // Declare device pointers
    float *d_A, *d_B, *d_C;

    // Allocate device memory
    cudaMalloc(&d_A, size_A);
    cudaMalloc(&d_B, size_B);
    cudaMalloc(&d_C, size_C);


    // Kernel launch code
    dim3 blockDim(TILE_SIZE, TILE_SIZE);
    dim3 gridDim((N + TILE_SIZE - 1) / TILE_SIZE, (M + TILE_SIZE - 1) / TILE_SIZE);
    matrixMultiplyOptimized<<<gridDim, blockDim>>>(d_A, d_B, d_C, M, N, K);

    // Synchronize device
    cudaDeviceSynchronize();

    // Free device memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    // Check for any CUDA errors
    cudaError_t error = cudaGetLastError();
    if (error != cudaSuccess) {
        std::cerr << "CUDA error: " << cudaGetErrorString(error) << std::endl;
        return -1;
    }

    return 0;

}

Writing t_matmul.cpp


In [ ]:
%%writefile nvtx_matmul.cpp
#include <cuda_runtime.h>
#include <nvtx3/nvToolsExt.h>
#include <iostream>

#define BLOCK_SIZE 16

__global__ void matrixMulKernel(float* A, float* B, float* C, int N) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    float sum = 0.0f;

    if (row < N && col < N) {
        for (int i = 0; i < N; i++) {
            sum += A[row * N + i] * B[i * N + col];
        }
        C[row * N + col] = sum;
    }
}

void matrixMul(float* A, float* B, float* C, int N) {
    nvtxRangePush("Matrix Multiplication");

    float *d_A, *d_B, *d_C;
    int size = N * N * sizeof(float);

    nvtxRangePush("Memory Allocation");
    cudaMalloc(&d_A, size);
    cudaMalloc(&d_B, size);
    cudaMalloc(&d_C, size);
    nvtxRangePop();

    nvtxRangePush("Memory Copy H2D");
    cudaMemcpy(d_A, A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, size, cudaMemcpyHostToDevice);
    nvtxRangePop();

    dim3 threadsPerBlock(BLOCK_SIZE, BLOCK_SIZE);
    dim3 numBlocks((N + BLOCK_SIZE - 1) / BLOCK_SIZE, (N + BLOCK_SIZE - 1) / BLOCK_SIZE);
    // timing code
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);


    nvtxRangePush("Kernel Execution");
    cudaEventRecord(start);
    matrixMulKernel<<<numBlocks, threadsPerBlock>>>(d_A, d_B, d_C, N);
    cudaEventRecord(stop);
    cudaDeviceSynchronize();
    nvtxRangePop();

    // Calculate elapsed time
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);
    std::cout << "Kernel execution time: " << milliseconds << " ms" << std::endl;

    // Clean up events
    cudaEventDestroy(start);
    cudaEventDestroy(stop);


    nvtxRangePush("Memory Copy D2H");
    cudaMemcpy(C, d_C, size, cudaMemcpyDeviceToHost);
    nvtxRangePop();

    nvtxRangePush("Memory Deallocation");
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    nvtxRangePop();

    nvtxRangePop();  // End of Matrix Multiplication
}

int main() {
    const int N = 1024;
    float *A = new float[N*N];
    float *B = new float[N*N];
    float *C = new float[N*N];

    // Initialize matrices A and B (for example, fill with 1.0f)
    for (int i = 0; i < N * N; i++) {
        A[i] = 1.0f;
        B[i] = 1.0f;
    }

    matrixMul(A, B, C, N);

    // (Optional) Check one value from C
    std::cout << "C[0] = " << C[0] << std::endl;

    delete[] A;
    delete[] B;
    delete[] C;

    return 0;
}


Overwriting nvtx_matmul.cpp


In [ ]:
%%writefile nvtx_matmul.cpp
#include <cuda_runtime.h>
#include <nvtx3/nvToolsExt.h>
#include <iostream>

#define BLOCK_SIZE 16

// Naive matrix multiplication kernel (same as your original)
__global__ void matrixMulKernelNaive(float* A, float* B, float* C, int N) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    float sum = 0.0f;

    if (row < N && col < N) {
        for (int i = 0; i < N; i++) {
            sum += A[row * N + i] * B[i * N + col];
        }
        C[row * N + col] = sum;
    }
}

// Optimized matrix multiplication kernel using tiling and shared memory
__global__ void matrixMulKernelOptimized(float* A, float* B, float* C, int N) {
    __shared__ float tileA[BLOCK_SIZE][BLOCK_SIZE];
    __shared__ float tileB[BLOCK_SIZE][BLOCK_SIZE];

    int row = blockIdx.y * BLOCK_SIZE + threadIdx.y;
    int col = blockIdx.x * BLOCK_SIZE + threadIdx.x;
    float sum = 0.0f;

    // Loop over tiles
    for (int m = 0; m < (N + BLOCK_SIZE - 1) / BLOCK_SIZE; m++) {
        // Load elements into shared memory
        if (row < N && m * BLOCK_SIZE + threadIdx.x < N)
            tileA[threadIdx.y][threadIdx.x] = A[row * N + m * BLOCK_SIZE + threadIdx.x];
        else
            tileA[threadIdx.y][threadIdx.x] = 0.0f;

        if (col < N && m * BLOCK_SIZE + threadIdx.y < N)
            tileB[threadIdx.y][threadIdx.x] = B[(m * BLOCK_SIZE + threadIdx.y) * N + col];
        else
            tileB[threadIdx.y][threadIdx.x] = 0.0f;

        __syncthreads();

        // Multiply the two tiles
        for (int k = 0; k < BLOCK_SIZE; k++) {
            sum += tileA[threadIdx.y][k] * tileB[k][threadIdx.x];
        }
        __syncthreads();
    }

    if (row < N && col < N) {
        C[row * N + col] = sum;
    }
}

void matrixMul(float* A, float* B, float* C, int N) {
    nvtxRangePush("Matrix Multiplication");

    float *d_A, *d_B, *d_C_naive, *d_C_opt;
    int size = N * N * sizeof(float);

    nvtxRangePush("Memory Allocation");
    cudaMalloc(&d_A, size);
    cudaMalloc(&d_B, size);
    cudaMalloc(&d_C_naive, size);
    cudaMalloc(&d_C_opt, size);
    nvtxRangePop();

    nvtxRangePush("Memory Copy H2D");
    cudaMemcpy(d_A, A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, size, cudaMemcpyHostToDevice);
    nvtxRangePop();

    dim3 threadsPerBlock(BLOCK_SIZE, BLOCK_SIZE);
    dim3 numBlocks((N + BLOCK_SIZE - 1) / BLOCK_SIZE, (N + BLOCK_SIZE - 1) / BLOCK_SIZE);

    // ----- Naive Kernel Execution -----
    cudaEvent_t startNaive, stopNaive;
    cudaEventCreate(&startNaive);
    cudaEventCreate(&stopNaive);

    nvtxRangePush("Naive Kernel Execution");
    cudaEventRecord(startNaive);
    matrixMulKernelNaive<<<numBlocks, threadsPerBlock>>>(d_A, d_B, d_C_naive, N);
    cudaEventRecord(stopNaive);
    cudaDeviceSynchronize();
    nvtxRangePop();

    float naiveTime = 0;
    cudaEventElapsedTime(&naiveTime, startNaive, stopNaive);
    std::cout << "Naive Kernel execution time: " << naiveTime << " ms" << std::endl;
    cudaEventDestroy(startNaive);
    cudaEventDestroy(stopNaive);

    // ----- Optimized Kernel Execution -----
    cudaEvent_t startOpt, stopOpt;
    cudaEventCreate(&startOpt);
    cudaEventCreate(&stopOpt);

    nvtxRangePush("Optimized Kernel Execution");
    cudaEventRecord(startOpt);
    matrixMulKernelOptimized<<<numBlocks, threadsPerBlock>>>(d_A, d_B, d_C_opt, N);
    cudaEventRecord(stopOpt);
    cudaDeviceSynchronize();
    nvtxRangePop();

    float optTime = 0;
    cudaEventElapsedTime(&optTime, startOpt, stopOpt);
    std::cout << "Optimized Kernel execution time: " << optTime << " ms" << std::endl;
    cudaEventDestroy(startOpt);
    cudaEventDestroy(stopOpt);

    // Copy one result (here, from the optimized kernel) back to host
    nvtxRangePush("Memory Copy D2H");
    cudaMemcpy(C, d_C_opt, size, cudaMemcpyDeviceToHost);
    nvtxRangePop();

    nvtxRangePush("Memory Deallocation");
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C_naive);
    cudaFree(d_C_opt);
    nvtxRangePop();

    nvtxRangePop();  // End of Matrix Multiplication
}

int main() {
    const int N = 1024;
    float *A = new float[N * N];
    float *B = new float[N * N];
    float *C = new float[N * N];

    // Initialize matrices A and B (for example, fill with 1.0f)
    for (int i = 0; i < N * N; i++) {
        A[i] = 1.0f;
        B[i] = 1.0f;
    }

    matrixMul(A, B, C, N);

    // (Optional) Check one value from C (result from optimized kernel)
    std::cout << "C[0] = " << C[0] << std::endl;

    delete[] A;
    delete[] B;
    delete[] C;

    return 0;
}


Writing nvtx_matmul.cpp


In [ ]:
!mv nvtx_matmul.cpp nvtx_matmul.cu

In [ ]:
!nvcc -O3 -o nvtx_matmul nvtx_matmul.cu



In [ ]:
!./nvtx_matmul


Naive Kernel execution time: 8.23997 ms
Optimized Kernel execution time: 0.00384 ms
C[0] = 0


In [ ]:
## Now let me try and write out the kernels myself (well w a bit of help 😅)

%%writefile tile_matmul1.cpp
#include <cuda_runtime.h>
#include <nvtx3/nvToolsExt.h>
#include <iostream>


__global__ void tiled_sq_matmul(float* A, float* B, float* C,  int N)
{

  // Defining local variables regardubg this thread
    int by = blockIdx.y;
    int bx = blockIdx.x;
    int ty = threadIdx.y;
    int tx = threadIdx.x;

    // Output matrix C[i,j]

    int i = blockDim.y*by + ty;
    int j = blockDim.x*bx + tx;

    // Allocating shared memory
    __shared__ float sh_A[TILE_WIDTH][TILE_WIDTH]
    __shared__ float sh_B[TILE_WIDTH][TILE_WIDTH]

    // Parallel mat mul
    float value = 0;
    // Splitting data into smaller tiles
    for (int phase = 0; N/TILE_WIDTH; phase++)
    {
      // load tiles into shared memory
      sh_A[ty][tx] = A[(i)*N+phase*TILE_WIDTH+tx];
      sh_B[ty][tx] = B[(phase*TILE_WIDTH +ty)*N+J];
      __syncthreads(); // ensuring all threads finish at the same time
      // Dot product is performed with elements on the shared memory
      for (int k =0; k < TILE_WIDTH; k++)
        value += sh_A[ty][k] * sh_B[k][tx];
      __syncthreads(); // ensuring all threads finish at the same time
    }

    // Storing result in Matrix C
    C[i*N+j] = value;


}




Writing tile_matmul1.cpp


In [ ]:
!mv tile_matmul1.cpp tile_matmul1.cu

In [ ]:
!nvcc -O3 -o tile_matmul1 tile_matmul1.cu


tile_matmul1.cu(21): error: identifier "TILE_WIDTH" is undefined
      __attribute__((shared)) float sh_A[TILE_WIDTH][TUKE_WIDTH]
                                         ^

tile_matmul1.cu(21): error: identifier "TUKE_WIDTH" is undefined
      __attribute__((shared)) float sh_A[TILE_WIDTH][TUKE_WIDTH]
                                                     ^

tile_matmul1.cu(22): error: expected a ";"
      __attribute__((shared)) float sh_B[TILE_WIDTH][TUKE_WIDTH]
                              ^

tile_matmul1.cu(25): warning #12-D: parsing restarts here after previous syntax error
      float value = 0;
                     ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

tile_matmul1.cu(31): error: identifier "sh_B" is undefined
        sh_B[ty][tx] = B[(phase*TILE_WIDTH +ty)*N+J];
        ^

tile_matmul1.cu(31): error: identifier "J" is undefined
        sh_B[ty][tx] = B[(phase*TILE_WIDTH +ty)*N+J];
                                                  ^



In [ ]:
%%writefile t_mm2.cpp
#include <cuda_runtime.h>
#include <nvtx3/nvToolsExt.h>
#include <iostream>
#include <cstdlib>


__global__ void tiled_sqr_matmul()
{
  for(int i = 0; i < N * N; i++)
  {
      m[i] = rand() % 100; // this will give us rand nums > 100
  }


}

// Init a sq matrix with rand nums
void init_matrix(float* A, int N)
{

}


int main() {
  // Set Matrix dims
  int N = 1 << 10; // N dim is 1 shifted over the times = 2^10
  size_t bytes = N * N * sizeof(float); // subbing in here float for int so lets keep that in mind

  // Allocate memory for matrix

  // Creating out pointers
  float* d_A;
  float* d_B;
  float* d_C;

  // Allocating memory
  cudaMallocManaged(&d_A, bytes);
  cudaMallocManaged(&d_B, bytes);
  cudaMallocManaged(&d_C, bytes);

  // Init our i/p matrices
  init_matrix(d_A, N);
  init_matrix(d_B, N);

  // Set our CTA and Grid sizes
  dim3 blockDim(TILE_WIDTH, TILE_WIDTH);
  dim3 gridDim((N + TILE_WIDTH - 1) / TILE_WIDTH, (N + TILE_WIDTH - 1) / TILE_WIDTH);







}



In [ ]:
!./tile_matmul1


In [ ]:
# Rename the source file
!mv t_matmul.cpp t_matmul.cu

# Compile for multiple architectures (covers T4, A100, V100)
!nvcc -gencode arch=compute_75,code=sm_75 \
      -gencode arch=compute_80,code=sm_80 \
      -gencode arch=compute_70,code=sm_70 \
      t_matmul.cu -o t_matmul



In [ ]:
# Run the executable
!./t_matmul

### Atomics

### In a gist --
You can think of atomics as a very fast, hardware-level mutex operation. It's as if each atomic operation does this:

1. `lock(memory_location)`
2. `old_value = *memory_location`
3. `*memory_location = old_value + increment`
4. `unlock(memory_location)`
5. `return old_value`

In [ ]:

%%writefile atomics.cpp
#include <cuda_runtime.h>
#include <stdio.h>

#define NUM_THREADS 1000
#define NUM_BLOCKS 1000

// Kernel without atomics (incorrect)
__global__ void incrementCounterNonAtomic(int* counter) {
    // not locked
    int old = *counter;
    int new_value  = old + 1;
    // not unlocked
    *counter = new_value;
}

// Kernel with atomics (correct)
__global__ void incrementCounterAtomic(int* counter) {
  int a = atomicAdd(counter, 1);
}

int main() {
    int h_counterNonAtomic = 0;
    int h_counterAtomic = 0;
    int *d_counterNonAtomic, *d_counterAtomic;

    // Allocate device memory
    cudaMalloc((void**)&d_counterNonAtomic, sizeof(int));
    cudaMalloc((void**)&d_counterAtomic, sizeof(int));

    // Copy initial counter values to device
    cudaMemcpy(d_counterNonAtomic, &h_counterNonAtomic, sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_counterAtomic, &h_counterAtomic, sizeof(int), cudaMemcpyHostToDevice);

    // Launch kernels
    incrementCounterNonAtomic<<<NUM_BLOCKS, NUM_THREADS>>>(d_counterNonAtomic);
    incrementCounterAtomic<<<NUM_BLOCKS, NUM_THREADS>>>(d_counterAtomic);

    // Copy results back to host
    cudaMemcpy(&h_counterNonAtomic, d_counterNonAtomic, sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(&h_counterAtomic, d_counterAtomic, sizeof(int), cudaMemcpyDeviceToHost);

    // Print results
    printf("Non-atomic counter value: %d\n", h_counterNonAtomic);
    printf("Atomic counter value: %d\n", h_counterAtomic);

    // Free device memory
    cudaFree(d_counterNonAtomic);
    cudaFree(d_counterAtomic);

    return 0;
}

Writing atomics.cpp


In [ ]:
%%writefile atomics.cpp
#include <cuda_runtime.h>
#include <stdio.h>
#include <stdlib.h>

#define NUM_THREADS 1000
#define NUM_BLOCKS 1000

#define CHECK_CUDA(call) {                                   \
    cudaError_t err = call;                                  \
    if (err != cudaSuccess) {                                \
        fprintf(stderr, "CUDA error in %s (%s:%d): %s\n",    \
            #call, __FILE__, __LINE__, cudaGetErrorString(err)); \
        exit(1);                                           \
    }                                                      \
}

// Kernel without atomics (incorrect due to race conditions)
__global__ void incrementCounterNonAtomic(int* counter) {
    int temp = *counter;
    temp = temp + 1;
    *counter = temp;
}

// Kernel with atomics (correct)
__global__ void incrementCounterAtomic(int* counter) {
    atomicAdd(counter, 1);
}

int main() {
    int h_counterNonAtomic = 0;
    int h_counterAtomic = 0;
    int *d_counterNonAtomic, *d_counterAtomic;

    // Allocate device memory
    CHECK_CUDA(cudaMalloc((void**)&d_counterNonAtomic, sizeof(int)));
    CHECK_CUDA(cudaMalloc((void**)&d_counterAtomic, sizeof(int)));

    // Copy initial counter values to device
    CHECK_CUDA(cudaMemcpy(d_counterNonAtomic, &h_counterNonAtomic, sizeof(int), cudaMemcpyHostToDevice));
    CHECK_CUDA(cudaMemcpy(d_counterAtomic, &h_counterAtomic, sizeof(int), cudaMemcpyHostToDevice));

    // Launch kernels
    incrementCounterNonAtomic<<<NUM_BLOCKS, NUM_THREADS>>>(d_counterNonAtomic);
    CHECK_CUDA(cudaGetLastError());  // Check for launch errors
    incrementCounterAtomic<<<NUM_BLOCKS, NUM_THREADS>>>(d_counterAtomic);
    CHECK_CUDA(cudaGetLastError());

    // Ensure kernels have completed
    CHECK_CUDA(cudaDeviceSynchronize());

    // Copy results back to host
    CHECK_CUDA(cudaMemcpy(&h_counterNonAtomic, d_counterNonAtomic, sizeof(int), cudaMemcpyDeviceToHost));
    CHECK_CUDA(cudaMemcpy(&h_counterAtomic, d_counterAtomic, sizeof(int), cudaMemcpyDeviceToHost));

    // Print results
    printf("Non-atomic counter value: %d\n", h_counterNonAtomic);
    printf("Atomic counter value: %d\n", h_counterAtomic);

    // Free device memory
    CHECK_CUDA(cudaFree(d_counterNonAtomic));
    CHECK_CUDA(cudaFree(d_counterAtomic));

    return 0;
}


Writing atomics.cpp


In [ ]:
!mv atomics.cpp atomics.cu

In [ ]:
!nvcc -gencode=arch=compute_80,code=sm_80 -O3 -o atomics atomics.cu


### Better to use this cmd as its more specific to the GPU im using -- A100

In [ ]:
!nvcc -arch=sm_80 -O3 -o atomics atomics.cu


In [ ]:
!./atomics

Non-atomic counter value: 18
Atomic counter value: 1000000


In [ ]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
!nvidia-smi


Tue Feb 18 21:56:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             41W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

### Cuda Streams ⛰️💧💧

### You can think of streams as "river streams" where the direction of operations flows only forward in time (like a timeline).
###  For example:
* Copy some data over (time step 1),
*Then do some computation (time step 2),
*Then copy some data back (time step 3).


### We can have multiple streams at once in CUDA, and each stream can have its own timeline. This allows us to overlap operations and make better use of the GPU.

###  For example:
* When training a massive LLM it would be silly to spend a ton of time loading all the tokens in and out of the GPU.
* Streams allow us to move data around while also doing computation at all times. Streams introduce a software abstraction called "prefetching", which is a way to move data around before it is needed.
*This is a way to hide the latency of moving data around.

In [ ]:
%%writefile streams1.cpp
#include <cuda_runtime.h>
#include <stdio.h>

#define CHECK_CUDA_ERROR(val) check((val), #val, __FILE__, __LINE__) // error checking macros to make sure operationns
                                                                      // go through succesfully

template <typename T>
void check(T err, const char* const func, const char* const file, const int line) {
    if (err != cudaSuccess) {
        fprintf(stderr, "CUDA error at %s:%d code=%d(%s) \"%s\" \n", file, line, static_cast<unsigned int>(err), cudaGetErrorString(err), func);
        exit(EXIT_FAILURE);
    }
}

__global__ void vectorAdd(const float *A, const float *B, float *C, int numElements) {
    int i = blockDim.x * blockIdx.x + threadIdx.x;
    if (i < numElements) {
        C[i] = A[i] + B[i];
    }
}

int main(void) {
    int numElements = 50000;
    size_t size = numElements * sizeof(float);
    float *h_A, *h_B, *h_C;
    float *d_A, *d_B, *d_C;
    cudaStream_t stream1, stream2; // use cuda stream type --

    // Allocate host memory
    h_A = (float *)malloc(size);
    h_B = (float *)malloc(size);
    h_C = (float *)malloc(size);

    // Initialize host arrays
    for (int i = 0; i < numElements; ++i) {
        h_A[i] = rand() / (float)RAND_MAX;
        h_B[i] = rand() / (float)RAND_MAX;
    }

    // Allocate device memory
    CHECK_CUDA_ERROR(cudaMalloc((void **)&d_A, size));
    CHECK_CUDA_ERROR(cudaMalloc((void **)&d_B, size));
    CHECK_CUDA_ERROR(cudaMalloc((void **)&d_C, size));

    // Create streams
    CHECK_CUDA_ERROR(cudaStreamCreate(&stream1));
    CHECK_CUDA_ERROR(cudaStreamCreate(&stream2));

    // Copy inputs to device asynchronously
    CHECK_CUDA_ERROR(cudaMemcpyAsync(d_A, h_A, size, cudaMemcpyHostToDevice, stream1));
    CHECK_CUDA_ERROR(cudaMemcpyAsync(d_B, h_B, size, cudaMemcpyHostToDevice, stream2));

    // Launch kernels
    int threadsPerBlock = 256;
    int blocksPerGrid = (numElements + threadsPerBlock - 1) / threadsPerBlock;
    vectorAdd<<<blocksPerGrid, threadsPerBlock, 0, stream1>>>(d_A, d_B, d_C, numElements);

    // Copy result back to host asynchronously
    CHECK_CUDA_ERROR(cudaMemcpyAsync(h_C, d_C, size, cudaMemcpyDeviceToHost, stream1));

    // Synchronize streams
    CHECK_CUDA_ERROR(cudaStreamSynchronize(stream1));
    CHECK_CUDA_ERROR(cudaStreamSynchronize(stream2));

    // Verify result
    for (int i = 0; i < numElements; ++i) {
        if (fabs(h_A[i] + h_B[i] - h_C[i]) > 1e-5) {
            fprintf(stderr, "Result verification failed at element %d!\n", i);
            exit(EXIT_FAILURE);
        }
    }

    printf("Test PASSED\n");

    // Cleaning up resources
    CHECK_CUDA_ERROR(cudaFree(d_A));
    CHECK_CUDA_ERROR(cudaFree(d_B));
    CHECK_CUDA_ERROR(cudaFree(d_C));
    CHECK_CUDA_ERROR(cudaStreamDestroy(stream1));
    CHECK_CUDA_ERROR(cudaStreamDestroy(stream2));
    free(h_A);
    free(h_B);
    free(h_C);

    return 0;
}

Writing streams1.cpp


In [ ]:
!mv streams1.cpp streams1.cu

In [ ]:
!nvcc -arch=sm_80 -O3 -o streams1 streams1.cu


In [ ]:
!./streams1

Test PASSED
